In [1]:
import os
import random
import asyncio
import configparser
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.chat_history import (
    #BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)


config = configparser.ConfigParser()
config.read('../config.ini')

groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')




store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

async def Message_to_Model(model,session_id,prompt):

    conversation_history=get_session_history(session_id)

    conversation_history.add_message(HumanMessage(content=prompt))
    
    response=model.invoke(
        [SystemMessage(content="You are a Movie Story Writer"),
         HumanMessage(content=prompt)]
    )

    conversation_history.add_message(SystemMessage(content=response.content))
    
    return response.content
   

async def model1_cohere(actor,location,theme):
    session_id = "cohere_story"
    prompt = f"Create a Bollywood story starring {actor}, set in {location}, with a theme of {theme}."
    cohere_model = ChatCohere(model="command-r-plus")
    return await Message_to_Model(cohere_model, session_id, prompt)

async def model2_groq(actor, location, theme):
    session_id = "groq_story"
    prompt = f"Create a Hollywood story starring {actor}, set in {location}, with a theme of {theme}."
    groq_model = ChatGroq(model="llama3-8b-8192")
    return await Message_to_Model(groq_model, session_id, prompt)

    
def actor_selector():
    actor=["Amithabh Bachchan","Tom Cruise","Allu Arjun","Salman Khan","Robert Downey Jr"]
    return random.choice(actor)

def location_selector():
    locations = ["Norwey", "Kerala", "Dubai", "Chaina","Japan"]
    return random.choice(locations)

def theme_selector():
    themes = ["Adventure", "Horror", "Comedy", "Mystery","Sci-Fi"]
    return random.choice(themes)

actor=actor_selector()
location=location_selector()
theme=theme_selector()

print(f"actor is {actor}")
print(f"location is {location}")
print(f"theme is {theme}")    


cohere_story = await model1_cohere(actor, location, theme)
print(f"\nStory from Cohere Model:\n{cohere_story}\n")

groq_story = await model2_groq(actor, location, theme)
print(f"\nStory from Groq Model:\n{groq_story}\n")

actor is Allu Arjun
location is Dubai
theme is Sci-Fi

Story from Cohere Model:
Title: "Dubai Dreams"

Introduction:

Allu Arjun stars as Raj, a young and ambitious Indian expat living in Dubai. Raj works as a software engineer during the day and spends his nights pursuing his passion for robotics and artificial intelligence. He dreams of creating something groundbreaking that will change the world. One night, while working in his makeshift lab, he receives a mysterious email inviting him to a secret underground facility in the heart of Dubai. Intrigued, Raj sets out to uncover the secrets of this facility, unaware that his life is about to take a drastic turn.

Act I: The Underground Facility

Raj arrives at the designated location, an abandoned warehouse on the outskirts of the city. He is greeted by a group of enigmatic individuals who introduce themselves as part of a secret organization dedicated to advancing technology beyond the constraints of government regulations. Their leade